In [15]:
# ===================================================================
# Cell 1: Setup and Imports
# ===================================================================
import pandas as pd
import numpy as np
import os
import json
import time
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score
from scipy.stats import ks_2samp
from scipy.stats import entropy
from scipy.stats import lmoment

# --- NOTE ---
# Make sure the sompy library files are in the same directory
from folder.sompy_isom import SOMFactory
from folder.aux_fun import aux_fun

# --- Installation Note ---
# !pip install tabpfn sklearn pandas seaborn matplotlib tqdm sompy scipy
    
print("Setup Complete!")

Setup Complete!


In [16]:
# ===================================================================
# Cell 2: Intelligent Synthetic Data Generator using sompy
# ===================================================================
# This cell is copied from your original notebook.

def generate_isom_data_advanced(df_real, target_column, num_synthetic_total, strategy="decision_boundary", roi_percentile=20, boundary_width=0.25):
    """
    Generates synthetic data by starting from real data points and exploring towards 
    neighboring prototypes within the Region of Interest (RoI).
    """
    if num_synthetic_total <= 0:
        return pd.DataFrame(columns=df_real.columns)

    print(f"Generating {num_synthetic_total} ADVANCED synthetic samples (Strategy: {strategy})...")

    # --- 1. Prepare Data and Train iSOM (Same as before) ---
    X_real = df_real.drop(columns=[target_column])
    y_real = df_real[target_column]
    data_for_som = np.hstack((X_real.values, y_real.values.reshape(-1, 1)))
    component_names = list(df_real.columns)

    sm = SOMFactory.build(data_for_som, normalization='range', initialization='pca', component_names=component_names)
    sm.som_lininit()
    sm.train(request_id=f'isom_gen_advanced_{strategy}', verbose=None)
    
    af = aux_fun()
    codebook = sm.codebook.matrix

    # --- 2. Identify Region of Interest (RoI) (Same as before) ---
    if strategy == "decision_boundary":
        target_values = codebook[:, -1]
        lower_bound, upper_bound = 0.5 - boundary_width, 0.5 + boundary_width
        roi_node_indices = np.where((target_values >= lower_bound) & (target_values <= upper_bound))[0]
    elif strategy == "hitmap":
        hits = af.som_hits(sm, sm._data)
        hit_threshold = np.percentile(hits[hits > 0], roi_percentile) if len(hits[hits > 0]) > 0 else 0
        roi_node_indices = np.where(hits >= hit_threshold)[0]
    else:
        raise ValueError("Invalid strategy.")

    if len(roi_node_indices) == 0:
        print("Warning: No RoI nodes found. Using all prototypes as fallback.")
        roi_node_indices = np.arange(sm.codebook.nnodes)
    
    print(f"  > RoI contains {len(roi_node_indices)} prototypes.")

    # --- 3. REFINED Synthetic Data Generation ---
    synthetic_samples = []
    rng = np.random.default_rng(42)
    neighborhood_matrix = af.som_unit_neighs(sm)
    bmus = af.som_bmus(sm, data_for_som) # Get Best Matching Unit for each real point

    for _ in tqdm(range(num_synthetic_total), desc="  > Generating samples", leave=False):
        # a. Pick a random prototype from the RoI
        p1_idx = rng.choice(roi_node_indices)
        p1 = codebook[p1_idx]

        # b. Find a REAL data point mapped to this prototype
        real_points_mapped_to_p1_indices = np.where(bmus == p1_idx)[0]

        d_start_point_norm = None

        if len(real_points_mapped_to_p1_indices) > 0:
            # --- Strategy 1: Start from a real point (your original logic) ---
            d_real_idx = rng.choice(real_points_mapped_to_p1_indices)
            d_start_point_norm = sm._data[d_real_idx] # Use normalized real data
        else:
            # --- Strategy 2 (FALLBACK): Start from the prototype itself ---
            d_start_point_norm = p1 # Use the prototype p1 as the start point

        # c. Find a neighboring prototype also in the RoI
        neighbor_indices = np.where(neighborhood_matrix[p1_idx] == 1)[0]
        roi_neighbors = np.intersect1d(neighbor_indices, roi_node_indices)
        
        if len(roi_neighbors) > 0:
            p2_idx = rng.choice(roi_neighbors)
            p2 = codebook[p2_idx]
        else:
            p2_idx = rng.choice(roi_node_indices)
            p2 = codebook[p2_idx]

        # d. Create new point by moving from the start point towards the neighbor
        alpha = rng.random() 
        new_sample_norm = d_start_point_norm + alpha * (p2 - p1)
        
        # e. Add noise, denormalize, and assign class (Same as before)
        features_norm = new_sample_norm[:-1]
        noise = rng.normal(0, 0.05, size=features_norm.shape)
        features_norm_noisy = np.clip(features_norm + noise, 0, 1)

        denormalized_data_wrapper = np.array([np.append(features_norm_noisy, 0)])
        final_features = sm._normalizer.denormalize_by(data_for_som, denormalized_data_wrapper)[:, :-1]

        target_prob_norm = np.clip(new_sample_norm[-1], 0, 1)
        final_class = rng.binomial(1, target_prob_norm)

        synthetic_samples.append(np.append(final_features.flatten(), final_class))

    if len(synthetic_samples) == 0:
        print("Warning: No synthetic samples were generated.")
        return pd.DataFrame(columns=df_real.columns)

    final_synthetic_df = pd.DataFrame(synthetic_samples, columns=df_real.columns)
    final_synthetic_df[target_column] = final_synthetic_df[target_column].astype(int)

    return final_synthetic_df

In [17]:
# ===================================================================
# Cell 3: Helper Functions (Data Loading)
# ===================================================================
def read_data(path):
    _, ext = os.path.splitext(path)
    if ext == ".parquet": return pd.read_parquet(path)
    elif ext == ".csv": return pd.read_csv(path, index_col=0)

def load_data(base_path, dataset_name):
    with open(os.path.join(base_path, dataset_name, f"{dataset_name}.meta.json")) as f: meta = json.load(f)
    
    # --- FIX: Removed trailing \ before the " ---
    train_data = read_data(os.path.join(base_path, dataset_name, f"{dataset_name}.{meta['format']}"))
    
    # --- FIX: Removed trailing \ before the " ---
    test_path = os.path.join(base_path, dataset_name, f"{dataset_name}_test.{meta['format']}")
    
    if os.path.exists(test_path):
        print(f"Found dedicated test set for {dataset_name}.")
        test_data = read_data(test_path)
    else:
        print(f"No dedicated test set for {dataset_name}. Splitting will be handled by index file.")
        test_data = None
        
    return train_data, test_data, meta

def get_indices_for_repeat(base_path, dataset_name, repeat_idx):
    train_idx_file = os.path.join(base_path, dataset_name, "train_indices.parquet")
    train_idx_splits = pd.read_parquet(train_idx_file)
    col_name = train_idx_splits.columns[repeat_idx]
    return train_idx_splits[col_name].values

In [18]:
# ===================================================================
# Cell 4: Main Configuration
# ===================================================================
BASE_DATA_PATH = "data"
DATASETS_TO_RUN = [
    # "airfoil_cl"
    "airfoil_cl", "airfoil_cl_m", "framed_safety", "framed_validity", 
    "solar_hex", "welded_beam", "welded_beam_balanced"
]

# --- Parameters for this notebook ---
# We'll use 50% of the real data as the basis for comparison
REAL_FRACTION_TO_TEST = 0.5
# We'll generate an equal number of synthetic samples
SYNTHETIC_STRATEGY = "hitmap" # Strategy to test

# --- Output Directory ---
PLOT_OUTPUT_DIR = "fidelity_plots"
os.makedirs(PLOT_OUTPUT_DIR, exist_ok=True)
print(f"Plots will be saved to: {PLOT_OUTPUT_DIR}")

Plots will be saved to: fidelity_plots


In [19]:
# ===================================================================
# Cell 4.5: Jensen-Shannon Divergence Helper Function
# ===================================================================
def js_divergence(p, q):
    """
    Calculates the Jensen-Shannon Divergence using the Freedman-Diaconis rule
    for robust binning.
    """
    # 1. Define common bins using Freedman-Diaconis Rule
    combined_data = np.concatenate([p, q])
    
    # Calculate IQR
    q75, q25 = np.percentile(combined_data, [75, 25])
    iqr = q75 - q25
    
    # Calculate bin width
    bin_width = 2 * iqr * (len(combined_data) ** (-1/3))
    
    # Handle case where bin_width is 0 (e.g., constant data)
    if bin_width == 0:
        bin_width = np.std(combined_data) / 10  # Fallback
        if bin_width == 0:
            bin_width = 0.1 # Absolute fallback

    # Calculate number of bins
    data_range = combined_data.max() - combined_data.min()
    n_bins = max(1, int(np.ceil(data_range / bin_width)))
    
    # Clamp to a reasonable maximum to avoid memory errors
    n_bins = min(n_bins, 250) 
    
    min_val = combined_data.min()
    max_val = combined_data.max()
    bins = np.linspace(min_val, max_val, n_bins)

    # 2. Create Probability Mass Functions (PMFs) using histograms
    #    (Using density=False to get counts)
    p_hist, _ = np.histogram(p, bins=bins)
    q_hist, _ = np.histogram(q, bins=bins)
    
    # Normalize histograms to ensure they are valid PMFs
    p_pmf = p_hist / p_hist.sum()
    q_pmf = q_hist / q_hist.sum()
    
    # 3. Add a tiny epsilon to avoid log(0) errors
    epsilon = 1e-10
    p_pmf = p_pmf + epsilon
    q_pmf = q_pmf + epsilon

    # 4. Calculate the average distribution M
    m_pmf = 0.5 * (p_pmf + q_pmf)
    
    # 5. Calculate D_JS using Kullback-Leibler divergence (entropy)
    # [cite: 229] The paper uses base 2 logarithm for D_JS to be bounded between 0 and 1
    js_div = 0.5 * (entropy(p_pmf, m_pmf, base=2) + entropy(q_pmf, m_pmf, base=2))
    
    return js_div

In [20]:
# ===================================================================
# Cell 5 (REVISED): Main Loop with Paginated PDF for Univariate Plots
# ===================================================================
from matplotlib.backends.backend_pdf import PdfPages # Add this import

all_fidelity_results = []

for dataset_name in tqdm(DATASETS_TO_RUN, desc="Overall Dataset Progress"):
    print(f"\n===== Running Fidelity Tests for: {dataset_name} =====")
    
    # --- 1. Load and Prepare Data ---
    full_train_data, _, meta = load_data(BASE_DATA_PATH, dataset_name)
    y_column = meta["label"]
    if pd.api.types.is_float_dtype(full_train_data[y_column]):
        threshold = full_train_data[y_column].median()
        full_train_data[y_column] = (full_train_data[y_column] <= threshold).astype(int)
    X_columns = full_train_data.columns.drop(y_column)
    
    shuffled_indices = get_indices_for_repeat(BASE_DATA_PATH, dataset_name, 0)
    n_real = int(REAL_FRACTION_TO_TEST * len(full_train_data))
    
    if n_real < 10:
        print(f"  Skipping {dataset_name}: too few real samples ({n_real}).")
        continue

    train_indices = shuffled_indices[:n_real]
    test_indices = shuffled_indices[n_real:] 
    
    real_subset_for_training = full_train_data.loc[train_indices].copy()
    real_subset_for_testing = full_train_data.loc[test_indices].copy()

    # --- 2. Generate Synthetic Data ---
    n_synthetic = len(real_subset_for_testing) 
    synthetic_data = generate_isom_data_advanced(real_subset_for_training, y_column, n_synthetic, strategy=SYNTHETIC_STRATEGY)
    
    if synthetic_data.empty:
        print(f"  Skipping {dataset_name}: synthetic data generation failed.")
        continue
        
    print(f"  > Real training subset size: {len(real_subset_for_training)}")
    print(f"  > Real testing subset size: {len(real_subset_for_testing)}")
    print(f"  > Synthetic data size: {len(synthetic_data)}")

    # --- 3. Run Fidelity Tests ---
    
    # == Test 1: Univariate Distribution Comparison (KDE Plots & K-S Test & JS Divergence) ==
    print("  > Running Test 1: Univariate Distributions...")
    
    # --- NEW: Paginated PDF Setup ---
    PLOTS_PER_PAGE = 15 # Your 5x3 grid
    N_COLS = 3
    N_ROWS = 5
    
    univariate_pdf_path = os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_univariate_dists.pdf")
    with PdfPages(univariate_pdf_path) as pdf:
        
        # Create a new figure for the first page
        fig, axes = plt.subplots(N_ROWS, N_COLS, figsize=(N_COLS * 5, N_ROWS * 4))
        axes = axes.flatten()
        plot_count = 0

        for i, col in enumerate(X_columns):
            
            # --- Get the correct axis on the current page ---
            ax_idx = plot_count % PLOTS_PER_PAGE
            ax = axes[ax_idx]

            # --- Run Calculations ---
            ks_stat, p_value = ks_2samp(real_subset_for_testing[col], synthetic_data[col])
            js_div = js_divergence(real_subset_for_testing[col].values, synthetic_data[col].values)
            
            l_moments_real = lmoment(real_subset_for_testing[col])
            l_moments_synth = lmoment(synthetic_data[col])
            l_skew_real, l_kurt_real = l_moments_real[2], l_moments_real[3]
            l_skew_synth, l_kurt_synth = l_moments_synth[2], l_moments_synth[3]

            # --- Store Results ---
            all_fidelity_results.append({
                "dataset": dataset_name, "feature": col, "ks_statistic": ks_stat,
                "ks_p_value": p_value, "js_divergence": js_div, "l_skew_real": l_skew_real,
                "l_skew_synth": l_skew_synth, "l_kurt_real": l_kurt_real, "l_kurt_synth": l_kurt_synth
            })
            
            # --- Plot Data ---
            sns.kdeplot(real_subset_for_testing[col], ax=ax, label="Real (Unseen)", color="blue", fill=True, alpha=0.5)
            sns.kdeplot(synthetic_data[col], ax=ax, label="Synthetic", color="orange", fill=True, alpha=0.5)
            ax.set_title(f"{col}\nKS p-val: {p_value:.3f} | JS Div: {js_div:.3f}", fontsize=10)
            ax.legend()
            
            plot_count += 1
            
            # --- Check if we need to save the page and start a new one ---
            if plot_count > 0 and plot_count % PLOTS_PER_PAGE == 0 and i < len(X_columns) - 1:
                # We've filled a page
                fig.suptitle(f"{dataset_name}: Univariate Distributions (Page {plot_count // PLOTS_PER_PAGE})", fontsize=16, y=1.03)
                fig.tight_layout()
                pdf.savefig(fig, bbox_inches='tight')
                plt.close(fig)
                
                # Create a new, blank figure and axes for the next page
                fig, axes = plt.subplots(N_ROWS, N_COLS, figsize=(N_COLS * 5, N_ROWS * 4))
                axes = axes.flatten()

        # --- After loop, save the last (potentially partially-filled) page ---
        # Turn off any remaining unused axes
        for j in range(ax_idx + 1, len(axes)):
            axes[j].axis('off')
            
        fig.suptitle(f"{dataset_name}: Univariate Distributions (Page {plot_count // PLOTS_PER_PAGE + 1})", fontsize=16, y=1.03)
        fig.tight_layout()
        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)
        
    print(f"  > Univariate plots saved to: {univariate_pdf_path}")


    # == Test 2: Multivariate Correlation Comparison (Heatmaps) ==
    print("  > Running Test 2: Multivariate Correlations...")
    corr_real = real_subset_for_testing[X_columns].corr()
    corr_synth = synthetic_data[X_columns].corr()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    sns.heatmap(corr_real, ax=ax1, annot=False, cmap="vlag", vmin=-1, vmax=1)
    ax1.set_title("Real Data Correlation Matrix")
    sns.heatmap(corr_synth, ax=ax2, annot=False, cmap="vlag", vmin=-1, vmax=1)
    ax2.set_title("Synthetic Data Correlation Matrix")
    fig.suptitle(f"{dataset_name}: Feature Correlation Comparison", fontsize=16)
    fig.tight_layout()
    plt.savefig(os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_correlation_heatmap.png"), bbox_inches='tight')
    plt.close(fig)


    # == Test 3: Dimensionality Reduction (t-SNE Plot) ==
    print("  > Running Test 3: t-SNE Visualization...")
    real_subset_tsne = real_subset_for_testing[X_columns].copy()
    synth_data_tsne = synthetic_data[X_columns].copy()
    real_subset_tsne['source'] = 'Real'
    synth_data_tsne['source'] = 'Synthetic'
    combined_df = pd.concat([real_subset_tsne, synth_data_tsne])
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(combined_df[X_columns])
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(combined_df) - 1))
    tsne_results = tsne.fit_transform(features_scaled)
    combined_df['tsne-2d-one'] = tsne_results[:,0]
    combined_df['tsne-2d-two'] = tsne_results[:,1]
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x="tsne-2d-one", y="tsne-2d-two", hue="source", palette={"Real": "blue", "Synthetic": "orange"}, data=combined_df, legend="full", alpha=0.7)
    plt.title(f"{dataset_name}: t-SNE Visualization (Real vs. Synthetic)")
    plt.savefig(os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_tsne_plot.png"), bbox_inches='tight')
    plt.close()


print("\n\n===== ALL FIDELITY TESTS COMPLETE! =====")

# --- Save the final CSV report ---
fidelity_results_df = pd.DataFrame(all_fidelity_results)
fidelity_results_df.to_csv(os.path.join(PLOT_OUTPUT_DIR, "all_fidelity_results.csv"), index=False)
print(f"Fidelity test results saved to {PLOT_OUTPUT_DIR}/all_fidelity_results.csv")

Overall Dataset Progress:   0%|          | 0/7 [00:00<?, ?it/s]


===== Running Fidelity Tests for: airfoil_cl =====
No dedicated test set for airfoil_cl. Splitting will be handled by index file.
Generating 332 ADVANCED synthetic samples (Strategy: hitmap)...
  > RoI contains 76 prototypes.


  > Generating samples:   0%|          | 0/332 [00:00<?, ?it/s]

  > Real training subset size: 554
  > Real testing subset size: 332
  > Synthetic data size: 332
  > Running Test 1: Univariate Distributions...
  > Univariate plots saved to: fidelity_plots\airfoil_cl_univariate_dists.pdf
  > Running Test 2: Multivariate Correlations...
  > Running Test 3: t-SNE Visualization...

===== Running Fidelity Tests for: airfoil_cl_m =====
No dedicated test set for airfoil_cl_m. Splitting will be handled by index file.
Generating 332 ADVANCED synthetic samples (Strategy: hitmap)...
  > RoI contains 56 prototypes.


  > Generating samples:   0%|          | 0/332 [00:00<?, ?it/s]

...

  > Real training subset size: 250
  > Real testing subset size: 250
  > Synthetic data size: 250
  > Running Test 1: Univariate Distributions...
  > Univariate plots saved to: fidelity_plots\solar_hex_univariate_dists.pdf
  > Running Test 2: Multivariate Correlations...
  > Running Test 3: t-SNE Visualization...

===== Running Fidelity Tests for: welded_beam =====
Found dedicated test set for welded_beam.
Generating 1000 ADVANCED synthetic samples (Strategy: hitmap)...
  > RoI contains 98 prototypes.


  > Generating samples:   0%|          | 0/1000 [00:00<?, ?it/s]

  > Real training subset size: 1000
  > Real testing subset size: 1000
  > Synthetic data size: 1000
  > Running Test 1: Univariate Distributions...
  > Univariate plots saved to: fidelity_plots\welded_beam_univariate_dists.pdf
  > Running Test 2: Multivariate Correlations...
  > Running Test 3: t-SNE Visualization...

===== Running Fidelity Tests for: welded_beam_balanced =====
Found dedicated test set for welded_beam_balanced.
Generating 1250 ADVANCED synthetic samples (Strategy: hitmap)...
  > RoI contains 110 prototypes.


  > Generating samples:   0%|          | 0/1250 [00:00<?, ?it/s]

  > Real training subset size: 1250
  > Real testing subset size: 1250
  > Synthetic data size: 1250
  > Running Test 1: Univariate Distributions...
  > Univariate plots saved to: fidelity_plots\welded_beam_balanced_univariate_dists.pdf
  > Running Test 2: Multivariate Correlations...
  > Running Test 3: t-SNE Visualization...


===== ALL FIDELITY TESTS COMPLETE! =====
Fidelity test results saved to fidelity_plots/all_fidelity_results.csv


In [21]:
# ===================================================================
# Cell 6: Analyze Fidelity Report Card (K-S Test & JS Divergence)
# ===================================================================
import pandas as pd

pd.set_option('display.max_rows', None)

# Load the results
fidelity_report_df = pd.read_csv(os.path.join(PLOT_OUTPUT_DIR, "all_fidelity_results.csv"))

# Set your significance level for K-S test
alpha = 0.05

# --- 1. Find "Problematic" Features based on K-S Test ---
problem_features_ks = fidelity_report_df[fidelity_report_df['ks_p_value'] <= alpha]

if problem_features_ks.empty:
    print("✅ K-S Test: No features were statistically different from real data.")
else:
    print(f"❌ K-S Test: Found {len(problem_features_ks)} features that are statistically different (p <= {alpha}).")
    print("All problematic features sorted by K-S statistic:")
    print(problem_features_ks.sort_values('ks_statistic', ascending=False))

print("\n" + "="*50 + "\n")

# --- 2. NEW: Find Features with Highest Divergence (JS Divergence) ---
# This ranks features by how different their distribution *shapes* are.
print("--- JS Divergence Analysis ---")
print("All features with the largest distribution shape difference (higher is worse):")
print(fidelity_report_df.sort_values('js_divergence', ascending=False))

# --- 3. NEW: L-Moment Shape Fidelity Report ---
print("\n--- L-Moment Shape Fidelity Report ---")
fidelity_report_df['l_skew_diff'] = abs(fidelity_report_df['l_skew_real'] - fidelity_report_df['l_skew_synth'])
fidelity_report_df['l_kurt_diff'] = abs(fidelity_report_df['l_kurt_real'] - fidelity_report_df['l_kurt_synth'])

l_moment_summary = fidelity_report_df.groupby('dataset').agg(
    avg_l_skew_diff=('l_skew_diff', 'mean'),
    avg_l_kurt_diff=('l_kurt_diff', 'mean')
).sort_values('avg_l_skew_diff')

print("Average Absolute Difference in L-Skew and L-Kurtosis (lower is better):")
print(l_moment_summary)

# --- 4. Enhanced Overall Dataset Fidelity Report ---
dataset_summary = fidelity_report_df.groupby('dataset').agg(
    percent_features_passed_ks=('ks_p_value', lambda p: (p > 0.05).mean() * 100),
    average_js_divergence=('js_divergence', 'mean'),
    average_l_skew_diff=('l_skew_diff', 'mean'),
    average_l_kurt_diff=('l_kurt_diff', 'mean')
).sort_values('average_js_divergence', ascending=True)

print("\n--- Enhanced Overall Dataset Fidelity Report ---")
print(dataset_summary)

pd.reset_option('display.max_rows')

❌ K-S Test: Found 284 features that are statistically different (p <= 0.05).
All problematic features sorted by K-S statistic:
                  dataset            feature  ks_statistic     ks_p_value  \
241       framed_validity  Material=Titanium      0.766445   0.000000e+00   
221         framed_safety  Material=Aluminum      0.685078  8.198266e-272   
198          airfoil_cl_m                x49      0.578313   1.090177e-51   
98             airfoil_cl                x49      0.545181   1.251667e-45   
0              airfoil_cl                 x0      0.536145   4.690741e-44   
100          airfoil_cl_m                 x0      0.533133   1.543405e-43   
229         framed_safety         SSB Offset      0.519373  1.168462e-149   
256       framed_validity               CS F      0.516630  4.835992e-165   
263       framed_validity          BB Length      0.514412  1.477991e-163   
267       framed_validity         SSB Offset      0.502587  9.091637e-156   
266       framed_validity 

In [22]:
# ===================================================================
# Cell 7: Compile Main Report (Corrected)
# ===================================================================
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
import numpy as np

print("Compiling PDF report...")

# --- Load the CSV from Cell 5 ---
try:
    fidelity_report_df = pd.read_csv(os.path.join(PLOT_OUTPUT_DIR, "all_fidelity_results.csv"))
except FileNotFoundError:
    print("Error: 'all_fidelity_results.csv' not found. Please run Cell 5 first.")
    # Or just exit if in a script
    raise

# ===================================================================
# START: ADDED ANALYSIS BLOCK (from Cell 6)
# ===================================================================
# This block calculates the '..._diff' columns that were missing
print("Running final analysis...")
if 'l_skew_real' in fidelity_report_df.columns:
    fidelity_report_df['l_skew_diff'] = abs(fidelity_report_df['l_skew_real'] - fidelity_report_df['l_skew_synth'])
    fidelity_report_df['l_kurt_diff'] = abs(fidelity_report_df['l_kurt_real'] - fidelity_report_df['l_kurt_synth'])
else:
    print("Warning: L-moment columns not found. Skipping L-moment analysis in report.")
    # Add empty columns so the rest of the script doesn't fail
    fidelity_report_df['l_skew_diff'] = 0
    fidelity_report_df['l_kurt_diff'] = 0

# Now, create the summary table using the newly calculated columns
dataset_summary = fidelity_report_df.groupby('dataset').agg(
    # percent_features_passed_ks=('ks_p_value', lambda p: (p > 0.05).mean() * 100),
    average_js_divergence=('js_divergence', 'mean'),
    average_l_skew_diff=('l_skew_diff', 'mean'),
    average_l_kurt_diff=('l_kurt_diff', 'mean')
).sort_values('average_js_divergence', ascending=True)

dataset_summary = dataset_summary.round(3) # Round for better display
# ===================================================================
# END: ADDED ANALYSIS BLOCK
# ===================================================================


# --- Create the PDF ---
with PdfPages(os.path.join(PLOT_OUTPUT_DIR, 'Fidelity_Report.pdf')) as pdf:

    # --- Page 1: Summary Table ---
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')
    ax.set_title('Overall Fidelity Report', fontsize=20, pad=20)
    
    # Use plt.table to render the DataFrame
    table = plt.table(cellText=dataset_summary.values,
                      colLabels=dataset_summary.columns,
                      rowLabels=dataset_summary.index,
                      loc='center',
                      cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    pdf.savefig(fig, bbox_inches='tight')
    plt.close(fig)

    # --- Subsequent Pages: One per dataset ---
    # Use dataset_summary.index to only loop over datasets that were actually run
    for dataset_name in dataset_summary.index:
        print(f"Adding report pages for {dataset_name}...")
        
        # --- Add a Title Page for the dataset ---
        fig_title, ax_title = plt.subplots(figsize=(11, 8.5))
        ax_title.axis('off')
        ax_title.text(0.5, 0.5, f'Dataset:\n{dataset_name}', ha='center', va='center', fontsize=30)
        pdf.savefig(fig_title)
        plt.close(fig_title)
        
        # --- Add t-SNE Plot ---
        tsne_path = os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_tsne_plot.png")
        if os.path.exists(tsne_path):
            fig_tsne, ax_tsne = plt.subplots(figsize=(11, 8.5))
            ax_tsne.set_title(f"{dataset_name} - t-SNE Plot", fontsize=16)
            img = plt.imread(tsne_path)
            ax_tsne.imshow(img)
            ax_tsne.axis('off')
            pdf.savefig(fig_tsne)
            plt.close(fig_tsne)
        
        # --- Add Correlation Heatmap ---
        corr_path = os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_correlation_heatmap.png")
        if os.path.exists(corr_path):
            fig_corr, ax_corr = plt.subplots(figsize=(11, 8.5))
            ax_corr.set_title(f"{dataset_name} - Correlation Heatmaps", fontsize=16)
            img_corr = plt.imread(corr_path)
            ax_corr.imshow(img_corr)
            ax_corr.axis('off')
            pdf.savefig(fig_corr)
            plt.close(fig_corr)

print(f"Report saved to {os.path.join(PLOT_OUTPUT_DIR, 'Fidelity_Report.pdf')}")

Compiling PDF report...
Running final analysis...
Adding report pages for solar_hex...
Adding report pages for welded_beam...
Adding report pages for welded_beam_balanced...
Adding report pages for airfoil_cl...
Adding report pages for airfoil_cl_m...
Adding report pages for framed_validity...
Adding report pages for framed_safety...
Report saved to fidelity_plots\Fidelity_Report.pdf


In [23]:
# ===================================================================
# Cell 8: Merge All PDF Reports
# ===================================================================
import os
from PyPDF2 import PdfMerger
# PLOT_OUTPUT_DIR = "fidelity_plots"
# DATASETS_TO_RUN = [
#     # "airfoil_cl"
#     "airfoil_cl", "airfoil_cl_m", "framed_safety", "framed_validity", 
#     "solar_hex", "welded_beam", "welded_beam_balanced"
# ]
print("Merging all PDF reports...")

# --- 1. Define the final output file ---
final_output_pdf = os.path.join(PLOT_OUTPUT_DIR, 'Complete_Fidelity_Report.pdf')

# --- 2. Create a list of all PDFs to merge ---
pdfs_to_merge = []

# First, add the main summary report
main_report_pdf = os.path.join(PLOT_OUTPUT_DIR, 'Fidelity_Report.pdf')
if os.path.exists(main_report_pdf):
    pdfs_to_merge.append(main_report_pdf)
else:
    print(f"Warning: Main report not found at {main_report_pdf}")

# Next, add all the individual univariate reports
for dataset_name in DATASETS_TO_RUN:
    univ_pdf_path = os.path.join(PLOT_OUTPUT_DIR, f"{dataset_name}_univariate_dists.pdf")
    if os.path.exists(univ_pdf_path):
        pdfs_to_merge.append(univ_pdf_path)

# --- 3. Perform the merge ---
if len(pdfs_to_merge) > 0:
    merger = PdfMerger()

    for pdf_file in pdfs_to_merge:
        print(f"  > Appending: {pdf_file}")
        merger.append(pdf_file)

    merger.write(final_output_pdf)
    merger.close()
    
    print(f"\n✅ Success! Merged {len(pdfs_to_merge)} PDFs into:")
    print(final_output_pdf)
else:
    print("❌ No PDF files were found to merge.")

Merging all PDF reports...
  > Appending: fidelity_plots\Fidelity_Report.pdf
  > Appending: fidelity_plots\airfoil_cl_univariate_dists.pdf
  > Appending: fidelity_plots\airfoil_cl_m_univariate_dists.pdf
  > Appending: fidelity_plots\framed_safety_univariate_dists.pdf
  > Appending: fidelity_plots\framed_validity_univariate_dists.pdf
  > Appending: fidelity_plots\solar_hex_univariate_dists.pdf
  > Appending: fidelity_plots\welded_beam_univariate_dists.pdf
  > Appending: fidelity_plots\welded_beam_balanced_univariate_dists.pdf

✅ Success! Merged 8 PDFs into:
fidelity_plots\Complete_Fidelity_Report.pdf
